# Vectorization and Feature Extraction

#### Import the Necessary libraries

In [1]:
#pandas and numpy for df manipulation
import pandas as pd
import numpy as np
import re
import nltk
import statistics

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#counter
from collections import Counter

#Preprocessing: tokenization, lemmatization, and stop words removal
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from textblob import TextBlob
lemmatizer = WordNetLemmatizer()
sent_tokenizer = PunktSentenceTokenizer()

#Vectorization
from nltk.probability import FreqDist #nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer()
bow_vectorizer = CountVectorizer()

#ngrams
from nltk.util import ngrams #nltk

# stopwords
# import nltk
nltk.download('stopwords')


#Classification and Metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import ClassifierChain
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics
#%pip install scikit-multilearn
#import

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### Import the preprocessed data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read the CSV file into a DataFrame
data = pd.read_csv('/content/drive/MyDrive/NOVA IMS/Text Mining Project/text_train_btp.csv')


In [ ]:
data.head(3)

In [ ]:
#X = data.drop('tag', axis=1)  # Exclude the 'tag' column
#y = data['tag']  # Store only the 'tag' column

#X['lyrics'][0]

# np.random.seed(42)

# # Get the number of rows in the original DataFrame
# num_rows_original = len(data)

# # Calculate the number of rows you want in the new DataFrame (half of the original)
# num_rows_new = 400

# # Randomly select half of the rows
# random_indices = np.random.choice(num_rows_original, size=num_rows_new, replace=False)

# # Create a new DataFrame with the randomly selected rows
# new_dataframe = data.iloc[random_indices]

# # Reset the index of the new DataFrame
# new_dataframe = new_dataframe.reset_index(drop=True)

# # Now 'new_dataframe' contains half of the rows randomly selected from 'original_dataframe'
# new_dataframe.head()

#### Shortening the dataset... a ver se isto corre #sendhelp

In [4]:
data_big, data_small, data_big_y, data_small_y = train_test_split(data['lyrics_filtered_string'], data['tag'], test_size=0.04, random_state=42, stratify=data['tag'])


In [ ]:
data_small

27629    come againits long since goneyou gonna see now...
32313    lovelyuuu ooh uuufirst moment make stayand say...
28158    verse 1theres much fuzz applevery little hair ...
67859    introhoneywoodsixyes chuckschucky hit drum dro...
82213    intro blackbearyeah ayy ayy ayy ayybeartrap so...
                               ...                        
52431    wondering shadows time walls reason condemned ...
82185    chorusvow suntest manvow sunwhen warriors begu...
86967    beforewe shine shine oni believe life spirit l...
43695    verseuhh ridin along black regalblack tint rep...
50608    years monthsand dayssince first time saw naked...
Name: lyrics_filtered_string, Length: 5399, dtype: object

In [5]:
X_train, X_val, y_train, y_val = train_test_split(data_small, data_small_y, test_size=0.2, random_state=42)

In [ ]:
X_train

53083     intro lady kaylan moriahweve come today meet d...
19459     separate tables sit eatin separate bedrooms sl...
22467     sit drawinga portrait dadi really visit himbef...
82113     introshould sit couchon lefthooklove boy got l...
17161     sorry call cannot completed dialed please use ...
                                ...                        
48778     love girlfinest girl worldi didnt knowi could ...
71834     every morning wake feeling cryingif drugs woul...
68199     baby could nightrisk risk wanna risk itrisk ri...
130951    black dots dripping mathblack inki thinkhunche...
75843     verse 1whose arms intowhose arms intowhose arm...
Name: lyrics_filtered_string, Length: 4319, dtype: object

In [6]:
train_lyr = X_train.to_list()
test_lyr = X_val.to_list()

#### Extracting the lyrics

In [ ]:
#lyr = data_small['lyrics'].to_list()
#lyr_f = data['lyrics_filtered_string'].to_list()
#lyr_fd = data['lyrics_string_fdist'].to_list()

#lyr

#### Bag of Words + Remove stopwords

In [ ]:
# def tokenized_frequency_vector(list_of_strings):
#     '''Takes as input a list of strings of text and returns a list of dictionaries representing the frequency of each tokenized word'''

#     # Create a tokenizer that only captures letters and numbers
#     tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

#     # Initialize an empty list to store the frequency vectors
#     frequency_vectors = []

#     # Process each string in the input list
#     for text in list_of_strings:

#         # Tokenize the text data - converted to string to avoid issues with numbers or other data types
#         tokens = tokenizer.tokenize(str(text))

#         # Convert the tokens to lowercase
#         tokens = [token.lower() for token in tokens]

#         # Remove stop words
#         #1. Create a set of stop words
#         stop_words = set(nltk.corpus.stopwords.words('english')) #CODE HERE

#         #remove stop words from the tokenized text
#         tokens = [token for token in tokens if token not in stop_words] #CODE HERE

#         # Create a frequency distribution for the tokenized text data
#         fdist = FreqDist(tokens)

#         # Append the frequency vector to the list of frequency vectors
#         frequency_vectors.append(dict(fdist))

#     return frequency_vectors

In [ ]:
# messages_freq_vectors = tokenized_frequency_vector(lyr)
# messages_freq_vectors

In [ ]:
# messages_freq_df_1 = pd.DataFrame(messages_freq_vectors)

# # Sort the columns by their sums in descending order
# sorted_cols = messages_freq_df_1.sum().sort_values(ascending=False).index

# # Reorder the columns of the DataFrame
# messages_freq_df_1 = messages_freq_df_1[sorted_cols]

# #this creates a sparse matrix
# messages_freq_df_1

#### Bag of Words

In [7]:
def count_vectorizer_to_df(train_lst_strs, test_lst_strs):

    # Create an instance of the CountVectorizer class - Default vectorizer does not remove stop words
    vectorizer = CountVectorizer()

    # Fit the vectorizer to the text data and transform the text data into a frequency matrix
    train_frequency_matrix = vectorizer.fit_transform(train_lst_strs)
    test_frequency_matrix = vectorizer.transform(test_lst_strs)


    # Convert the frequency matrix to a Pandas DataFrame
    train_df = pd.DataFrame(train_frequency_matrix.toarray(), columns=vectorizer.get_feature_names_out())
    test_df = pd.DataFrame(test_frequency_matrix.toarray(), columns=vectorizer.get_feature_names_out())

    return train_df, test_df

In [8]:
train_lyr_dtm, test_lyr_dtm = count_vectorizer_to_df(train_lyr, test_lyr)

# Sort the columns by their sums in descending order
train_sorted_cols = train_lyr_dtm.sum().sort_values(ascending=False).index
#test_sorted_cols = test_lyr_dtm.sum().sort_values(ascending=False).index

# Reorder the columns of the DataFrame
train_lyr_dtm = train_lyr_dtm[train_sorted_cols]
test_lyr_dtm = test_lyr_dtm[train_sorted_cols]

#this creates matrix where nans are already filled with 0s
train_lyr_dtm
#test_lyr_dtm

,like,dont,know,got,get,love,never,see,one,cant,...,hatersnever,hatersmy,hatersit,hatersi,hatershigh,hatersfor,hatersdo,haterscause,hatersand,zzzzzzzzzzzzdahduhdnnduhmurdoc
0,5,0,2,0,7,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,1,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,9,2,2,1,7,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4314,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4315,5,7,8,1,1,0,5,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4316,9,2,2,1,4,4,2,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4317,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=4319)
components = pca.fit_transform(train_lyr_dtm)

plt.plot(pca.explained_variance_ratio_.cumsum(), marker='o')
plt.axhline(y=0.8, color='r', linestyle='-')
plt.show()

In [ ]:
#messages_dtm.to_csv('example.csv', index=False)

In [9]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(train_lyr_dtm, y_train)

MultinomialNB()

In [10]:
# Make predictions on the test set
y_pred = model.predict(test_lyr_dtm)

In [11]:
from sklearn.metrics import accuracy_score, classification_report
# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Optionally, print more detailed classification metrics
print(classification_report(y_val, y_pred))

Accuracy: 0.59
              precision    recall  f1-score   support

     country       0.00      0.00      0.00        22
        misc       0.50      0.38      0.43        47
         pop       0.58      0.77      0.66       443
         rap       0.63      0.89      0.74       318
          rb       0.00      0.00      0.00        51
        rock       0.50      0.02      0.03       199

    accuracy                           0.59      1080
   macro avg       0.37      0.34      0.31      1080
weighted avg       0.54      0.59      0.51      1080



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### One Hot Encoding

In [ ]:
def oh_count_vectorizer_to_df(list_of_strings):

    # Create an instance of the CountVectorizer class - Default vectorizer does not remove stop words
    vectorizer = CountVectorizer(binary = True)

    # Fit the vectorizer to the text data and transform the text data into a onehot encoded matrix
    ohe_matrix = vectorizer.fit_transform(list_of_strings)

    # Convert the frequency matrix to a Pandas DataFrame
    df = pd.DataFrame(ohe_matrix.toarray(), columns=vectorizer.get_feature_names_out()) #get_feature_names_out()

    return df

In [ ]:
messages_dtm_one_hot = oh_count_vectorizer_to_df(lyr)

#check if there any rows with any value greater than 1
messages_dtm_one_hot.max(axis=1).value_counts()

1    2700
dtype: int64

#### TF-IDF

In [ ]:
def compute_tf(list_of_strings):
    '''Computes the term frequency for each word in a list of messages - TF looks at importance of a word in a document (message)'''

    # Create a tokenizer that only captures letters and numbers
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

    #set stop words
    stop_words = set(nltk.corpus.stopwords.words('english'))

    # Initialize an empty list to store the term frequencies
    tf = []

    for text in list_of_strings:

        # Tokenize the text data - converted to string to avoid issues with numbers or other data types
        tokens = tokenizer.tokenize(str(text))

        #convert to lowercase and remove stopwords
        tokens = [token.lower() for token in tokens]
        tokens = [token for token in tokens if token not in stop_words]

        # Count the occurrences of each token
        token_counts = Counter(tokens)

        # Compute the term frequency for each token in the message
        tf_dict = {token: count / len(tokens) for token, count in token_counts.items()}
        tf.append(tf_dict)

    return tf

In [ ]:
def compute_idf(list_of_strings):
    '''takes as input a list of messages and returns a dictionary containing the IDF for each unique word across all of the messages in the input list'''

    # Create a tokenizer that only captures letters and numbers
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

    #set stop words
    stop_words = set(nltk.corpus.stopwords.words('english'))

    # Initialize the IDF dictionary
    idf = {}

    #Step 1 - Compute the total number of documents
    n_docs = len(list_of_strings)

    #Step 2 - Count the number of documents containing each term
    for text in list_of_strings:

        # Tokenize the text data - converted to string to avoid issues with numbers or other data types
        tokens = tokenizer.tokenize(str(text))

        #convert to lowercase and remove stopwords
        tokens = [token.lower() for token in tokens]
        tokens = [token for token in tokens if token not in stop_words]

        #Count 1 occurrence of each unique word appearing in the message
        token_counts = Counter(set(tokens))

        # Update the IDF dictionary - add the counts for this document to the existing counts for the token
        for token, count in token_counts.items():
            idf[token] = idf.get(token, 0) + count

    # Compute the IDF for each term - natural log of the quotient between the number of documents and the number of documents containing the term
    for token, count in idf.items():
        idf[token] = np.log(n_docs / count)
    return idf

In [ ]:
def compute_tfidf(list_of_strings):

    # Compute the term frequency for each message
    tf = compute_tf(list_of_strings)

    # Compute the inverse document frequency for each term in the corpus
    idf = compute_idf(list_of_strings)

    # Initialize the TF-IDF list
    tfidf = []

    # Compute the TF-IDF for each message - tf has the same length as the number ofmessages
    for tf_dict in tf:

        # Initialize an empty dictionary to store the TF-IDF values for this message
        tfidf_dict = {}

        # Compute the TF-IDF value for each token in the message
        for token, tf_val in tf_dict.items():
            tfidf_dict[token] = tf_val * idf[token]

        # Append the TF-IDF dictionary for this message to the list of dicts - each dict having a Token - TF-IDF value for all tokens in each message
        tfidf.append(tfidf_dict)

    return tfidf

In [ ]:
messages_tfidf_df = pd.DataFrame(compute_tfidf(lyr))
messages_tfidf_df.tail(10)

,get,high,horsedont,make,laughi,dont,think,youhappy,divorcei,even,...,knowwont,comingill,showmy,whiletell,heavyhearted,toowell,shouldtell,alivefor,worthplease,worthill
2690,0.007752,NaN,NaN,NaN,NaN,0.011665,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2691,NaN,NaN,NaN,0.022262,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2693,0.009150,NaN,NaN,0.019430,NaN,0.022950,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2694,NaN,NaN,NaN,NaN,NaN,0.021350,0.014681,NaN,NaN,0.018234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2695,0.028638,NaN,NaN,NaN,NaN,0.021548,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2696,0.017474,NaN,NaN,0.005565,NaN,0.075600,0.020342,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2698,NaN,NaN,NaN,NaN,NaN,0.027704,NaN,NaN,NaN,NaN,...,0.282179,0.282179,0.282179,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2699,0.009458,NaN,NaN,NaN,NaN,NaN,0.014681,NaN,NaN,NaN,...,NaN,NaN,NaN,0.072486,0.144973,0.072486,0.072486,0.072486,0.072486,0.072486


#### TF-IDF sklearn

In [ ]:
def tf_idf_to_df(list_of_strings):

    # Create an instance of the TdidfVectorized class - Default vectorizer does not remove stop words
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer to the text data and transform the text data into a frequency matrix
    frequency_matrix = vectorizer.fit_transform(list_of_strings)

    # Convert the frequency matrix to a Pandas DataFrame
    df = pd.DataFrame(frequency_matrix.toarray(), columns=vectorizer.get_feature_names_out()) #get_feature_names_out()

    return df

In [ ]:
messages_tfidf_sklearn = tf_idf_to_df(lyr)

#check descriptive statistics of df - allows us to see how sparse the matrix is
messages_tfidf_sklearn.describe()